In [1]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
import glob
import shutil
import pprint 
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data
import time
import pprint as pp
import collections
import scipy


pp = pprint.PrettyPrinter()

Using TensorFlow backend.


In [2]:
# Create Image Directory from Train.bson
# Set Download to False if directory has already been created, otherwise True

Download = False

start = time.time()

if Download:
    
    data = bson.decode_file_iter(open('/media/raj/96661FA2661F81E3/DOWNLOADS/Cdiscount_bson_files/train.bson', 'rb'))
    prod_to_category = dict()
    picture_pipeline = []
    Cat_Count = 0

    for c,d in enumerate(data):
        picture=[]
        category_id = d['category_id']
        number_pics = len(d['imgs'])
        prod_to_category[category_id] = number_pics


        if number_pics >3 and Cat_Count<250:
            #print Cat_Count
            Cat_Count +=1
            directory = './../Cdiscount2/Images'+str(category_id)+'/'
            if not os.path.exists(directory):
                os.makedirs(directory)
                for e, pic in enumerate(d['imgs']):
                    picture = imread(io.BytesIO(pic['picture']))
                    scipy.misc.imsave(str(directory)+str(category_id)+str(e)+'.png', picture)
                    #print type(picture)
                    #io.imsave(str(category_id)+str(e),picture)        
        else:
            pass


    print len(prod_to_category)
    print len(set(prod_to_category.keys()))

    for i,v in prod_to_category.items():
        if v>4:
            print i,v

    x = collections.Counter(prod_to_category.values())
    print x

In [3]:
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return files,targets


In [4]:
def Move_Images_png(From_dir,Folder_name,Num_images):
    From_path = From_dir + "*.png" if From_dir[-1]=="/" else From_dir + "/*.png"
    To_path = From_dir.replace("Train",Folder_name)
    To_path = To_path+"/" if To_path[-1]!="/" else To_path
    if not os.path.exists(To_path):
        os.makedirs(To_path)
    x = glob.glob(From_path)
    for i in x[0:Num_images-1]:
        shutil.move(i,To_path+str(i.split("/")[-1]))

In [5]:
# If Move is set to True, move a single image into the test folder

Move = False

if Move:
    for i in glob.glob("./../Cdiscount2/Train/*/"):
        Move_Images_png(i,'Test',2)

In [6]:
# If Augmentation is set to True, start image Augmentation in the training set

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import matplotlib.pyplot as plt

Augmentation = False

if Augmentation:
    Pre_Aug_Files, Pre_Aug_Labels = load_dataset("./../Cdiscount2/Train/")
    Category_folder=[]
    for i in Pre_Aug_Files:
        Category_folder.append(i.split("/")[-2])
    print len(Category_folder)
    
    datagen = ImageDataGenerator(rotation_range=90,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
    
    for i in range(len(Pre_Aug_Files)):
        img = load_img(Pre_Aug_Files[i])  # this is a PIL image
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
        
        if not os.path.exists("./../Cdiscount2/Train/"+str(Category_folder[i])):
            os.makedirs("./../Cdiscount2/Train/"+str(Category_folder[i]))
            
        count=0
        pointer = 4
        for batch in datagen.flow(x, batch_size=1,save_to_dir="./../Cdiscount2/Train/"+str(Category_folder[i]),save_prefix=str(Category_folder[i])+"_aug", save_format='png'):
            pointer += 1
            count += 1
            if count > 20:
                break  # otherwise the generator would loop indefinitely

In [7]:
#Creating Validation sets

Move1 = False

if Move1:
    for i in glob.glob("./../Cdiscount2/Train/*/"):
        Move_Images_png(i,'Validation',6)

In [8]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(180, 180))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [9]:
# load train, test, and validation datasets

train_files, train_targets = load_dataset('./../Cdiscount2/Train')
valid_files, valid_targets = load_dataset('./../Cdiscount2/Validation')
test_files, test_targets = load_dataset('./../Cdiscount2/Test')


In [10]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 49/49 [00:00<00:00, 514.43it/s]


In [11]:
print train_tensors.shape
#print train_targets[0]

(2980, 180, 180, 3)


In [12]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

In [13]:
from keras.callbacks import ModelCheckpoint  

train_from_scratch = False

if train_from_scratch== True:
    
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=2,padding='same',activation='relu',input_shape=(180,180,3)))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Conv2D(filters=32, kernel_size=2,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Conv2D(filters=64, kernel_size=2,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(49, activation='softmax'))
    ### TODO: Define your architecture.
    
    model.summary()
    
    model.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy'])
    
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(train_tensors)

    epochs = 10


    if not os.path.exists("./saved_models_20_rmsprop"):
        os.makedirs("./saved_models_20_rmsprop")
    checkpointer = ModelCheckpoint(filepath='saved_models_20_rmsprop/weights.best.from_scratch.hdf5', 
                                   verbose=1, save_best_only=True)


    # fits the model on batches with real-time data augmentation as well
    model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=32),
                        validation_data=(valid_tensors, valid_targets),
                        steps_per_epoch=len(train_tensors) / 1, epochs=epochs,
                        callbacks=[checkpointer],verbose=1)


In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K


datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        shear_range=0.2,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

datagen.fit(train_tensors)

# creating the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# Adding a fully-connected layer
x = Dense(1024, activation='relu')(x)
# Adding a logistic layer -- For the 49 output classes
predictions = Dense(49, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

# Training only the top layers, freezing up the base layers of the Inception Model
for layer in base_model.layers:
    layer.trainable = False

# compile the model
model.compile(optimizer='Nadam', loss='categorical_crossentropy',metrics=['accuracy'])

# Training the model for a few epochs
epochs = 20


if not os.path.exists("./saved_models_20_inception"):
    os.makedirs("./saved_models_20_inception")
checkpointer = ModelCheckpoint(filepath='saved_models_20_inception/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)


model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=32),
                    validation_data=(valid_tensors, valid_targets),
                    steps_per_epoch=len(train_tensors) / 1, epochs=epochs,
                    callbacks=[checkpointer],verbose=1)



Epoch 1/20
   6/2980 [..............................] - ETA: 6800s - loss: 5.0316 - acc: 0.0677

In [19]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

(0, 'input_2')
(1, 'conv2d_101')
(2, 'batch_normalization_95')
(3, 'activation_95')
(4, 'conv2d_102')
(5, 'batch_normalization_96')
(6, 'activation_96')
(7, 'conv2d_103')
(8, 'batch_normalization_97')
(9, 'activation_97')
(10, 'max_pooling2d_11')
(11, 'conv2d_104')
(12, 'batch_normalization_98')
(13, 'activation_98')
(14, 'conv2d_105')
(15, 'batch_normalization_99')
(16, 'activation_99')
(17, 'max_pooling2d_12')
(18, 'conv2d_109')
(19, 'batch_normalization_103')
(20, 'activation_103')
(21, 'conv2d_107')
(22, 'conv2d_110')
(23, 'batch_normalization_101')
(24, 'batch_normalization_104')
(25, 'activation_101')
(26, 'activation_104')
(27, 'average_pooling2d_10')
(28, 'conv2d_106')
(29, 'conv2d_108')
(30, 'conv2d_111')
(31, 'conv2d_112')
(32, 'batch_normalization_100')
(33, 'batch_normalization_102')
(34, 'batch_normalization_105')
(35, 'batch_normalization_106')
(36, 'activation_100')
(37, 'activation_102')
(38, 'activation_105')
(39, 'activation_106')
(40, 'mixed0')
(41, 'conv2d_116')
(42

In [34]:
# we chose to train the top few layers. I will freeze the first 249 layers.

for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

epochs=1
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])


model.fit(train_tensors, train_targets, batch_size=64,
                    validation_data=(valid_tensors, valid_targets),
                    epochs=epochs,
                    callbacks=[checkpointer],verbose=1)

Train on 2980 samples, validate on 245 samples
Epoch 1/1
2980/2980 [==============================] - 277s - loss: 7.0969e-06 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000


In [14]:
# Load model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
# creating the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# Adding a fully-connected layer
x = Dense(1024, activation='relu')(x)
# Adding a logistic layer -- For the 49 output classes
predictions = Dense(49, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

# Training only the top layers, freezing up the base layers of the Inception Model
for layer in base_model.layers:
    layer.trainable = False

# compile the model
model.compile(optimizer='Nadam', loss='categorical_crossentropy',metrics=['accuracy'])
model.load_weights("./saved_models_20_inception/weights.best.from_scratch.hdf5")

# Testing robustness of the model on multiple validation sets

from sklearn.model_selection import train_test_split
import numpy as np

scores = []

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(valid_tensors, valid_targets, test_size=0.1, random_state=i)
    #print X_test.shape, y_test.shape
    Object_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in X_test]
    test_accuracy = 100*np.sum(np.array(Object_predictions)==np.argmax(y_test, axis=1))/len(Object_predictions)
    scores.append(test_accuracy)
    print('Test accuracy: %.2f%%' % test_accuracy)


print("%.2f%% (+/- %.2f%%)" % (np.mean(scores), np.std(scores)))


Test accuracy: 96.00%
Test accuracy: 100.00%
Test accuracy: 96.00%
Test accuracy: 92.00%
Test accuracy: 92.00%
Test accuracy: 100.00%
Test accuracy: 96.00%
Test accuracy: 96.00%
Test accuracy: 92.00%
Test accuracy: 92.00%
95.20% (+/- 2.99%)


In [35]:
# Predicting the output categories for the unseen test set
Object_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# reporting test accuracy
test_accuracy = 100*np.sum(np.array(Object_predictions)==np.argmax(test_targets, axis=1))/len(Object_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)



Test accuracy: 69.0000%
